## Compute Welfares

In [1]:
# Define the various scenarios
N = 100
experiments = [
    dict(name='ces', param='theta', range=(4,8,N)),
    dict(name='exponential', param='alpha', range=(1,10,N)),
    dict(name='translog', param='alpha', range=(1,10,N))
]

In [2]:
def compute_case(expm):
    '''Evaluates welfare for the planner and the competitive equilibrium for various values of a specified parameter.
    
    Parameters
    ----------
    expm: dict
        Description of the experiment to perform. Contains several fields:
        - name: name of the experiment, the corresponding model file is expected to be model_*name*.yaml
        - param: name of the parameter to change
        - range: set of parameter values (min value, max value, N values)
    Returns
    -------
    df: pandas.DataFrame
        Each line contains:
        - parameter value
        - welfare for the planner optimum
        - welfare for the competitive equilibrium
    '''
        
    
    import pandas
    from dolo import yaml_import
    from dolo.algos.fg.steady_state import find_deterministic_equilibrium
    from dolo.algos.fg.perturbations_higher_order import approximate_controls
    from numpy import exp, array, linspace, column_stack

    
    case = expm['name']
    param = expm['param']
    range = expm['range']
    
    # create parameter range
    parm_vec = linspace(*range)
        
    # import model and get discount rate
    model = yaml_import('model_{}.yaml'.format(case))
    beta = model.get_calibration('beta')
    
    values_ce = []
    values_po = []
    
    # solve all calibrations
    for i in parm_vec:
        
        # Set calibrated parater value
        pp = {param:i}
        model.set_calibration(**pp)
        
        # Solve for the competitive equilibrium
        model.set_calibration(po=0)
        calib = find_deterministic_equilibrium(model)
        dr_ce = approximate_controls(model, order=2, steady_state=calib, verbose=False)

        # Solve for the social planner equilibrium
        model.set_calibration(po=1)
        calib = find_deterministic_equilibrium(model)
        dr_po = approximate_controls(model, order=2, steady_state=calib, verbose=False)     
        
                    
            
        # Initial point at which to evaluate the welfare
        eval_point = calib['states'].copy()
        # Change initial value of N
        eval_point[1]*=0.5 # to 50 % of its of its steady-state value
#         eval_point[1]=1    # to 1

        # Evalueate welfare
        w_ce = dr_ce(eval_point)[1]
        w_po = dr_po(eval_point)[1]
        
        # Convert to consumption equivalent
        v_ce = exp((1-beta)*w_ce)
        v_po = exp((1-beta)*w_po)
        
        # Add to list of values
        values_ce.append(v_ce)
        values_po.append(v_po)
    
   
    # Construct dataframe
    columns = [h.format(case) for h in ['{}_'+param, '{}_po', '{}_ce']]
    df = pandas.DataFrame( column_stack([parm_vec, values_po, values_ce]), columns=columns)
    
    return df

In [3]:
# parrallel evaluation (needs 3 workers)
# from IPython.parallel import Client
# rc = Client()
# dviews = rc[:3]
# all_welfares = dviews.map_sync(compute_case, experiments)

In [ ]:
# serial evaluation
all_welfares = [compute_case(exp) for exp in experiments]

In [ ]:
# Add welfare gains to dataframe
import pandas
df = pandas.concat(all_welfares, axis=1)
# compute gains
for exp in experiments:
    case = exp['name']
    df['{}_gain'.format(case)] = (df['{}_po'.format(case)]/df['{}_ce'.format(case)]-1)*100
# save to Excel
df.to_excel('welfare_gains.xls')
df

## Make plots

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
fig = plt.figure(figsize=(16,3))
plt.subplot(131)
plt.plot(df['ces_theta'],df['ces_gain'])
plt.ylabel('Welfare gains (%)')
plt.xlabel("$\\theta$")
yl = plt.ylim()
plt.ylim(0,yl[1])
plt.title("CES-Benassy")
plt.grid()
# xlim(4,8)
# ylim(0,0.2)
plt.subplot(132)
plt.plot(df['exponential_alpha'],df['exponential_gain'])
plt.grid()
yl = plt.ylim()
plt.ylim(0,yl[1])
plt.title("Exponential")
plt.xlabel("$\\alpha$")
plt.subplot(133)
plt.plot(df['translog_alpha'],df['translog_gain'])
plt.grid()
yl = plt.ylim()
plt.ylim(0,yl[1])
plt.title("Translog")
plt.xlabel("$\\alpha$")
plt.savefig('BGM_welfare_gains.png')
plt.savefig('BGM_welfare_gains.pdf')
plt.savefig('BGM_welfare_gains.svg')


In [ ]:
## optional: export with plotly
from plotly import plotly
link = plotly.plot_mpl(fig, filename='BGM_welfare_gains')